In [ ]:
import importlib.util
import subprocess

def install_package(package):
    subprocess.check_call(["pip", "install", package])

def check_and_install(package):
    spec = importlib.util.find_spec(package)
    if spec is None:
        print(f"{package} is not installed. Installing...")
        install_package(package)
    else:
        print(f"{package} is already installed.")

# List of required packages
required_packages = [
    "pandas",
    "numpy",
    "spacy",
    "scipy",
    "nltk",
    "lexicalrichness",
    "readability",
    "vaderSentiment",
    "gensim",
    "sklearn",
    "json",
    "transformers",
    "torch",
    # "keras==2.15.0",
    # "tensorflow==2.15.0",
]

# Check and install required packages
for package in required_packages:
    check_and_install(package)

# Additional installation steps for specific packages
if not importlib.util.find_spec("en_core_web_lg"):
    print("Downloading 'en_core_web_lg' model for spaCy...")
    subprocess.check_call(["python", "-m", "spacy", "download", "en_core_web_lg"])

print("All required libraries are installed.")

import pandas as pd
import numpy as np
import spacy
import re
# from scipy.spatial import distance
# import math
# import pickle
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import string
from lexicalrichness import LexicalRichness
import readability
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# from gensim.models import KeyedVectors
# from gensim.similarities import WmdSimilarity
from sklearn.feature_extraction.text import CountVectorizer
import json
from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine
import torch
from keras.models import load_model
import pickle as pkl
from keras.preprocessing.sequence import pad_sequences
import unicodedata
import re
from joblib import dump, load
from keras.models import Sequential
from keras.layers import Dense, Embedding, Flatten, Dropout
# from keras.callbacks import EarlyStopping
import pandas as pd
import os

# Rebuild the model architecture
max_words = 10000
embedding_dim = 100
maxlen = 100
batch_size = 200
epochs = 20

pandas is already installed.
numpy is already installed.
spacy is already installed.
scipy is already installed.
nltk is already installed.
lexicalrichness is not installed. Installing...
readability is not installed. Installing...
vaderSentiment is not installed. Installing...
gensim is already installed.
sklearn is already installed.
json is already installed.
transformers is already installed.
torch is already installed.
All required libraries are installed.


In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Counting the number of empty annotated reviews folders
import os

count = 0

for paper in os.listdir("/content/drive/MyDrive/annotated_reviews"):
    if len(os.listdir(os.path.join("/content/drive/MyDrive/annotated_reviews", paper))) == 0:
        count += 1
        print(paper)

.ipynb_checkpoints


In [3]:
print(len(os.listdir("/content/drive/MyDrive/annotated_reviews")))

600


In [ ]:
dir = '/content/drive/MyDrive'

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Helper functions

In [ ]:
sarcasm_model = Sequential()
sarcasm_model.add(Embedding(max_words, embedding_dim, input_shape=(maxlen,)))
sarcasm_model.add(Flatten())
sarcasm_model.add(Dense(100, activation="relu"))
sarcasm_model.add(Dropout(0.3))
sarcasm_model.add(Dense(50, activation="relu"))
sarcasm_model.add(Dropout(0.3))
sarcasm_model.add(Dense(2, activation="softmax"))

# Compile the model
sarcasm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Load the model weights
sarcasm_model.load_weights(os.path.join(dir, 'sarcasm_detection_model_best_preprocessed.h5'))

# Load sarcasm detection model and tokenizer
# sarcasm_model = load_model('sarcasm_detection_model_best.h5', compile=False)
# with open('tokenizer.pkl', 'rb') as f:
#     sarcasm_tokenizer = pkl.load(f)
sarcasm_tokenizer = load(os.path.join(dir, 'tokenizer.joblib'))
from keras.preprocessing.text import Tokenizer
import tensorflow as tf



In [ ]:
def load_glove_embeddings(file_path):
    embeddings = {}
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

nlp = spacy.load("en_core_web_lg")

glove_file_path = "/content/drive/MyDrive/glove.6B/glove.6B.100d.txt"  # Path to GloVe embeddings file
embeddings1 = load_glove_embeddings(glove_file_path)

# Code to Get confidence score for a given sentence
conf_model_name = "bert-base-uncased"
conf_tokenizer = BertTokenizer.from_pretrained(conf_model_name)
conf_model = BertForSequenceClassification.from_pretrained(conf_model_name, num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=d1300a91a8eabc38c1f94dcf2232ab400a3706d91820df885c8e9970856e5b32
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from num2words import num2words

In [ ]:
import re
def convert_numbers_to_words(sentence):
    # Remove periods and semicolons
    # cleaned_sentence = sentence.replace(".", "").replace(";", "")

    # Split the sentence into words
    words = sentence.split()

    removed_symbols = [",", ":", ";", ".", "?", "!", "$", "(", ")", "@", "#", "%", "*", "+", "=", "^", "[", "]", "\\", "/", "&", "`", "~", "<", ">", "_", "{", "}"]

    # Convert numbers to words
    for i, word in enumerate(words):
        # Remove commas and colons from the word
        num_str = re.sub(r"[\[\]^,:-;.?!$()@#%*+=\\/&`~<>_{}]", "", word)
        try:
            num = int(num_str)
            words[i] = num2words(num)
        except ValueError:
            pass
    # Join the words back into a sentence
    converted_sentence = " ".join(words)

    # Add back the periods and semicolons
    for i in removed_symbols:
        converted_sentence = converted_sentence.replace(f" {i}", i)
    # converted_sentence = converted_sentence.replace(" ,", ",").replace(" .", ".").replace(" ;", ";").replace(" -", "-").replace(" ?", "?").replace(" !", "!").replace(" $", "$").replace(" (", "(").replace(" )", ")")

    return converted_sentence

In [ ]:
sample_sent = "(14) (15) -1 823. *72 456% $33 @2 82! 99? ^343 [23 2] \\454 /44 {8 99} `223 ~23 &6 _88_ <89 >00 0> 99< these are some unclean numbers."
convert_numbers_to_words(sample_sent)

'fourteen fifteen minus one eight hundred and twenty-three seventy-two four hundred and fifty-six thirty-three two eighty-two ninety-nine three hundred and forty-three twenty-three two four hundred and fifty-four forty-four eight ninety-nine two hundred and twenty-three twenty-three six eighty-eight eighty-nine zero zero ninety-nine these are some unclean numbers.'

In [ ]:
# def convert_numbers_to_words(sentence):
#     words = sentence.split()  # Split the sentence into words
#     converted_words = []

#     for word in words:
#         if word.isdigit():  # Check if the word is a numeric value
#             converted_word = num2words(int(word))  # Convert the numeric word to words
#             converted_words.append(converted_word)
#         else:
#             converted_words.append(word)  # Keep non-numeric words unchanged

#     return ' '.join(converted_words)  # Join the words back into a sentence

def convert_numbers_to_words(sentence):
    # Remove periods and semicolons
    # cleaned_sentence = sentence.replace(".", "").replace(";", "")

    # Split the sentence into words
    words = sentence.split()

    removed_symbols = [",", ":", ";", ".", "?", "!", "$", "(", ")", "@", "#", "%", "*", "+", "=", "^", "[", "]", "\\", "/", "&", "`", "~", "<", ">", "_", "{", "}"]

    # Convert numbers to words
    for i, word in enumerate(words):
        # Remove commas and colons from the word
        num_str = re.sub(r"[\[\]^,:-;.?!$()@#%*+=\\/&`~<>_{}]", "", word)
        try:
            num = int(num_str)
            words[i] = num2words(num)
        except ValueError:
            pass
    # Join the words back into a sentence
    converted_sentence = " ".join(words)

    # Add back the periods and semicolons
    for i in removed_symbols:
        converted_sentence = converted_sentence.replace(f" {i}", i)
    # converted_sentence = converted_sentence.replace(" ,", ",").replace(" .", ".").replace(" ;", ";").replace(" -", "-").replace(" ?", "?").replace(" !", "!").replace(" $", "$").replace(" (", "(").replace(" )", ")")

    return converted_sentence

def pos_annotator(txt):
    # nlp = spacy.load("en_core_web_lg")
    doc = nlp(txt)
    for token in doc:
        if token.pos_ == 'VERB':
            return 1
    return 0

#  Function to detect Hedge
def hedge_detector(txt):
    l=['usually','to some extent','admittedly',"probably", "likely", "perhaps", "possibly", "maybe", "could", "may", "might", "seems", "appears", "potentially", "conceivably", 'presumably', 'supposedly', 'allegedly', 'apparently', 'purportedly', 'hypothetically']
    txt = txt.lower()
    for phrase in l:
        if phrase in txt:
            return 1
    return 0



# Writing Style Scores
#********Lexical Features********
# syllable count

def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word[index]=='e':
                count -= 1
    if count == 0:
        count += 1
    return count

def avg_syllable_count(text):
    tokens = sent_tokenize(text)
    l1 = [word for token in tokens for word in token.split()]
    print(tokens)
    print(l1)
    # print([syllable_count(word) for word in l1])
    return np.average([syllable_count(word) for word in l1])


#avg word length

def avg_wordlength(text):
    string1 = text.translate(string.punctuation)
    # print(string1)
    tokens = word_tokenize(string1, language='english')
    # print(tokens)
    st = [",", ".", "'", "!", '"', "#", "$", "%", "&", "(", ")", "*", "+", "-", ".", "/", ":", ";", "<", "=", '>', "?",
          "@", "[", "\\", "]", "^", "_", '`', "{", "|", "}", '~', '\t', '\n']
    stop = stopwords.words('english') + st
    words = [word for word in tokens if word not in stop]
    return np.average([len(word) for word in words])

#avg sentence length by number of words

def avg_sentlengthbyword(text):
    tokens = sent_tokenize(text)
    return np.average([len(token.split()) for token in tokens])

def count_functionalwords(text):
    functional_words = """a between in nor some upon
    about both including nothing somebody us
    above but inside of someone used
    after by into off something via
    all can is on such we
    although cos it once than what
    am do its one that whatever
    among down latter onto the when
    an each less opposite their where
    and either like or them whether
    another enough little our these which
    any every lots outside they while
    anybody everybody many over this who
    anyone everyone me own those whoever
    anything everything more past though whom
    are few most per through whose
    around following much plenty till will
    as for must plus to with
    at from my regarding toward within
    be have near same towards without
    because he need several under worth
    before her neither she unless would
    behind him no should unlike yes
    below i nobody since until you
    beside if none so up your
    """

    functional_words = functional_words.split()
    tokens = sent_tokenize(text)
    wo_punc = [text.translate(str.maketrans ('', '', string.punctuation)) for text in tokens]
    l1 = [word for token in wo_punc for word in token.split()]
    count = 0

    for i in l1:
        if i in functional_words:
            count += 1

    return count / len(l1)

#punc count

def punctuation_count_ratio(text):
    st = [",", ".", "'", "!", '"', ";", "?", ":", ";"]
    count = 0
    for i in text:
        if (i in st):
            count = count + 1
    text = text.replace(" ", "")
    # print(text)
    return float(count) / float(len(text))


#********Readability score********


#https://github.com/burrsettles/readability/blob/master/readability.py

def remove_newlines(text):
    cleaned_text = re.sub(r'\n', '', text)
    return cleaned_text

#*******Lexical diversity*********

def style_score(text):
  #lexical features
  avg_syllable_count0 = avg_syllable_count(text)
  avg_wordlength0 = avg_wordlength(text)
  avg_sentlengthbyword0 = avg_sentlengthbyword(text)
  count_functionalwords0 = count_functionalwords(text)
  punctuation_count_ratio0 = punctuation_count_ratio(text)

  #readability scores
  results = readability.getmeasures(text, lang='en')
  fleschreadingease = results['readability grades']['FleschReadingEase']
  kincaid = results['readability grades']['Kincaid']
  gunningfogindex = results['readability grades']['GunningFogIndex']
  dalechallindex = results['readability grades']['DaleChallIndex']

  #Vocab richness
  lex = LexicalRichness(text)
  yulek = lex.yulek
  herdan = lex.Herdan
  maas = lex.Maas
  msttr = lex.msttr(segment_window=3)
  mattr = lex.mattr(window_size=3)
  simpsond = lex.simpsond


  vector = [avg_syllable_count0, avg_wordlength0, avg_sentlengthbyword0, count_functionalwords0, punctuation_count_ratio0,

            fleschreadingease, kincaid, gunningfogindex, dalechallindex,

            yulek, herdan, maas, msttr, mattr, simpsond
            ]
#   print(text, vector)

  return vector



# Function for Binary Sentiment Classification
def bin_sentiment(txt):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(txt)
    if sentiment['neu']>sentiment['pos'] and sentiment['neu']>sentiment['neg']:
        return 0
    else:
        return 1



# Contextualization Functions

# Word Mover's Distance

def word_mover_dist(text1, text2):
    tokens1 = nltk.word_tokenize(text1.lower())
    tokens2 = nltk.word_tokenize(text2.lower())

    # Remove out-of-vocabulary words
    tokens1 = [token for token in tokens1 if token in embeddings1]
    tokens2 = [token for token in tokens2 if token in embeddings1]

    # Calculate distance matrix
    distance_matrix = np.zeros((len(tokens1), len(tokens2)))
    for i, token1 in enumerate(tokens1):
        for j, token2 in enumerate(tokens2):
            distance_matrix[i, j] = cosine(embeddings1[token1], embeddings1[token2])

    # Solve Word Mover's Distance problem
    wmd = distance_matrix.mean()  # Mean distance as a simplified version

    return wmd


# Cosine Similarity

def cosine_similarity(text1, text2):
    # Vectorize the texts
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform([text1, text2])
    vec1 = X.toarray()[0]
    vec2 = X.toarray()[1]

    # Calculate cosine similarity
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    similarity = dot_product / (norm_vec1 * norm_vec2)
    return similarity

# Jaccard Similarity

def jaccard_similarity(text1, text2):
    # Tokenize texts
    tokens1 = set(text1.split())
    tokens2 = set(text2.split())

    # Calculate Jaccard similarity
    intersection = len(tokens1.intersection(tokens2))
    union = len(tokens1.union(tokens2))
    similarity = intersection / union
    return similarity


# Pre-processing functions
def sentence_tokenization(txt):
    sentences = sent_tokenize(txt)
    return sentences

# Preprocessing functions for sarcasm detection
def remove_extra_whitespaces(text):
    '''
    Removes extra whitespaces from a string, if present

    Args:
        text (str): String to which the function is to be applied, string

    Returns:
        Clean string without extra whitespaces
    '''
    return re.sub(r'^\s*|\s\s*', ' ', text).strip()

def remove_accented_chars(text):
    '''
    Removes all accented characters from a string, if present

    Args:
        text (str): String to which the function is to be applied, string

    Returns:
        Clean string without accented characters
    '''
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')

def remove_special_characters(text):
    '''
    Removes all special characters from a string, except hyphens, single and double quotes, and punctuation marks, if present

    Args:
        text (str): String to which the function is to be applied, string

    Returns:
        Clean string without special characters
    '''
    # Remove special characters except hyphens, single and double quotes, and punctuation marks
    return re.sub(r'[^\w\s\-.,\'"]', '', text)

def to_lower_case(text):
    '''
    Converts text to lower case

    Args:
        text (str): String to which the function isd to be applied, string

    Returns:
        Text, converted to lower case
    '''
    return text.lower()

def complete_text_preprocessing(text):
    text = remove_extra_whitespaces(text)
    text = remove_accented_chars(text)
    text = remove_special_characters(text)
    text = to_lower_case(text)
    return text

def sarcasm_detection(txt):
    txt = complete_text_preprocessing(txt)
    txt = sarcasm_tokenizer.texts_to_sequences([txt])
    txt = pad_sequences(txt, maxlen=100)
    prediction = sarcasm_model.predict(txt)
    # print(prediction)
    return prediction[0][1]

def get_sentiment_confidence(statement):
    inputs = conf_tokenizer(statement, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = conf_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).squeeze().tolist()

    return max(probs[1], probs[0])

def remove_short_words(sentence):
    # Split the sentence into words
    words = sentence.split()

    # Filter out words with length less than 3 characters
    filtered_words = [word for word in words if len(word) >= 3]

    # Join the filtered words back into a sentence
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# Annotation

In [ ]:
import chardet
def get_encoding(file):
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        # print(result["encoding"])
    return result['encoding']

def read_json(file_path):
    with open(file_path, 'rb') as file:
        data = file.read().decode('utf-8-sig')
        data = json.loads(data)
    return data

In [ ]:
import os

folder_path = "/content/drive/MyDrive/annotated_reviews"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [ ]:
review_folder = "/content/drive/MyDrive/ICLR_2022"
paper_folder = "/content/drive/MyDrive/json"
annotation_folder = "/content/drive/MyDrive/annotated_reviews"

In [ ]:
part_papers_to_process = os.listdir(review_folder)
part_papers_to_process.sort()
print(len(part_papers_to_process))
print(part_papers_to_process)

1094
['-70L8lpp9DF', '-8sBpe7rDiV', '-AOEi-5VTU8', '-ApAkox5mp', '-Gk_IPJWvk', '-HSOjDPfhBJ', '-TSe5o7STVR', '-e4EXDWXnSn', '-llS6TiOew', '-ngwPqanCEZ', '-w2oomO6qgc', '01AMRlen9wJ', '03RLpj-tc_', '04pGUg0-pdZ', '06Wy2BtxXrz', '085y6YPaYjP', '0DLwqQLmqV', '0DcZxeWfOPt', '0EXmFzUn5I', '0JzqUlIVVDd', '0RDcd5Axok', '0RqDp8FCW5Z', '0UXT6PpRpW', '0cgU-BZp2ky', '0jP2n0YFmKG', '0kPL3xO4R5', '0no8Motr-zO', '0rcbOaoBXbg', '0sgntlpKDOz', '0xiJLKH-ufZ', '12RoR2o32T', '14F3fI6MGxX', '18Ys0-PzyPI', '1HxTO6CTkz', '1JDiK_TbV4S', '1L0C5ROtFp', '1NUsBU-7HAL', '1NvflqAdoom', '1W0z96MFEoH', '1ugNpm7W6E', '1wVvweK3oIb', '1xXvPrAshao', '1zwleytEpYx', '2-mkiUs9Jx7', '25kzAhUB1lz', '26gKg6x-ie', '28ib9tf6zhr', '2_vhkAMARk', '2bO2x8NAIMB', '2eXhNpHeW6E', '2f1z55GVQN', '2ggNjUisGyr', '2sDQwC_hmnM', '2t7CkQXNpuq', '31d5RLCUuXC', '3HJOA-1hb0e', '3ILxkQ7yElm', '3PN4iyXBeF', '3Pbra-_u76D', '3YqeuCVwy1d', '3eIrli0TwQ', '3jooF27-0Wy', '3pugbNqOh5m', '3rULBvOJ8D2', '3tbDrs77LJ5', '3wNcr5nq56', '3wU2UX0voE', '4-D6CZkR

In [ ]:
import time

# B3Nde6lvab (197:198)
# For above paper, Science Parse doesn't work properly, and hence, no sections are extracted
# Hence, barring it, else all done from 0 to 199
# Riya and Aditya, continue from id 200 (i.e., 200 to 399, and 400 to 599)

part_papers_to_process = os.listdir(review_folder)
part_papers_to_process.sort()
part_papers_to_process = part_papers_to_process[482:600]

for paper in part_papers_to_process:
    print(f"\nCurrent Paper: {paper}")
    if paper not in os.listdir(annotation_folder):
        os.makedirs(os.path.join(annotation_folder, paper))
    paper_jsons = os.listdir(paper_folder)
    paper_folder_name = [name for name in paper_jsons if paper in name][0]
    json_data = read_json(os.path.join(paper_folder, paper_folder_name))
    sections = json_data["metadata"]["sections"]
    paper_content = ""
    for section in sections:
        paper_content += section['text']
    docs = os.listdir(os.path.join(review_folder, paper))
    reviews = [x for x in docs if x[-4:] == '.txt' and not x.__contains__("MetaReview")]
    for review in reviews:
        if review not in os.listdir(os.path.join(annotation_folder, paper)):
            start = time.time()
            print(f"Current Review: {review}")
            review_path = os.path.join(review_folder, paper, review)
            content = ""
            with open(review_path, "r", encoding=get_encoding(review_path)) as file:
                content = file.read().replace("\n", "")
            review_sentences = sentence_tokenization(content)
            annotated_review = ""
            sent_count = 0
            for sentence in review_sentences:
                # print(f"Sentence: {sentence}")
                print(f"Current sentence number: {sent_count}")
                # print(f"Length of Sentence: {len(sentence.split())}")
                sentence = convert_numbers_to_words(sentence)
                sentence = remove_short_words(sentence)
                if len(sentence.split()) < 5:
                    # print("Skipping sentence as length is less than 5.")
                    continue
                annotations = [[],[],[],[],[],[],[]]
                annotations[0].append(pos_annotator(sentence))
                annotations[1].append(bin_sentiment(sentence))
                annotations[2].append(hedge_detector(sentence))
                list_of_contextualization = [None, None, None]
                # list_of_contextualization[0] = word_mover_dist(sentence, paper_content)
                list_of_contextualization[1] = cosine_similarity(sentence, paper_content)
                list_of_contextualization[2] = jaccard_similarity(sentence, paper_content)
                annotations[3] =  list_of_contextualization
                annotations[4].extend(style_score(sentence))
                annotations[5].append(sarcasm_detection(sentence))
                annotations[6].append(get_sentiment_confidence(sentence))
                annotated_review += sentence + str(annotations)
                sent_count += 1
            with open(os.path.join(annotation_folder, paper, review), "w") as text_file:
                text_file.write(annotated_review)
            end = time.time()
            print(f"Time required to annotate this review: {(end - start) * 1000} ms")

Streaming output truncated to the last 5000 lines.
['They used the resulting graph representation input GNNs, but there’s nothing that precludes the same input from being linearized(by standard graph walks) and used sequential models.Given the expressiveness the framework, was hoping for evaluation tasks that require more global and long-range kind properties.']
['They', 'used', 'the', 'resulting', 'graph', 'representation', 'input', 'GNNs,', 'but', 'there’s', 'nothing', 'that', 'precludes', 'the', 'same', 'input', 'from', 'being', 'linearized(by', 'standard', 'graph', 'walks)', 'and', 'used', 'sequential', 'models.Given', 'the', 'expressiveness', 'the', 'framework,', 'was', 'hoping', 'for', 'evaluation', 'tasks', 'that', 'require', 'more', 'global', 'and', 'long-range', 'kind', 'properties.']
1/1 [==============================] - 0s 40ms/step
Current sentence number: 12
['Unfortunately, most the paper focuses tasks that have local(intra-procedural) solution.']
['Unfortunately,', 'mos

In [ ]:
print(f"Number of reviews annotated: {len(os.listdir(annotation_folder))}")

Number of reviews annotated: 600
